<h1>Importing Required Modules</h1>

In [39]:
# Importing Required Modules
import glob
import csv

<h1>Get all Log files in the present working directory.</h1>

In [40]:
# List Of Files
Filenames = glob.glob("C:\PowerBI\Power_BI_Reports\DeepSecurity\SyslogCatchAll*.txt")


<h1>Read contents of all files into a Datum List.</h1>

In [41]:
# Read contents of file
Datum = list()
for filename in Filenames:
    File = open(filename)
    Datum.extend(File.readlines())
    

<h1> Events and their corressponding Signature IDs </h1> :
<ol>
    <li> System Events </li>
    <li> Security Events </li>
    <ul> 
        <li> Custom Logs =  40 </li>
        <li> Integrity Monitoring  = 2,000,000 -  2,999,999</li>
        <li> Log Inspection = 3,000,000 - 3,999,999 </li>
    </ul>
</ol>
   

In [42]:
CustomLog =  list ()
IntegrityMonitoring = list()
LogInspection = list()
Result = list()
Global  = list()

<h1> Parse each line of the file at one time.</h1>

In [43]:
for line in Datum:
    Data = list()
    IndexKeys = dict()
    split_pipe  = line.strip().split("|")
   
    
    # Extract Header Information
    Header = split_pipe[0]
    HeaderSplit = Header.split("\t")
    # Extract Log Generation Date
    value = HeaderSplit[0]
    value = value[0:value.find("TMDSSERVER")].strip()
    
    # Common Fields that are present in all logs 
    # ["Receiving Time","Generated Time","Service Facility","IPv4 Address","DeviceVendor","DeviceProduct","DeviceVersion","SignatureID","Name","Severity"]
    Data.append(value)
    Data.extend(HeaderSplit[:-1])
    Data.extend(split_pipe[1:7])
   
    # The event type can be determined by device Product and the signature id of the event
    # Security Events are generated by Deep Security Agent
    # System Events are generated by Deep Security Manager
    SignatureID = int(split_pipe[4])
    Extension = split_pipe[7]
    
    # The event is generated by Deep Security Agent
    if split_pipe[2] == "Deep Security Agent":
        
        # Custom Log Events
        if SignatureID == 40: 
            IndexKeys["cn1"] = Extension.find("cn1=")
            IndexKeys["cn1Label"] = Extension.find("cn1Label=")
            IndexKeys["dvc"]=Extension.find("dvc")
            IndexKeys["TrendMicroDsTenant"]=Extension.find("TrendMicroDsTenant")
            IndexKeys["TrendMicroDsTenantId"]=Extension.find("TrendMicroDsTenantId")
            IndexKeys["cs1"]=Extension.find("cs1=")
            IndexKeys["cs1Label"]=Extension.find("cs1Label=")
            IndexKeys["fname"]=Extension.find("fname=")
            IndexKeys["duser"]=Extension.find("duser=")
            IndexKeys["shost"]=Extension.find("shost=")
            IndexKeys["msg"]=Extension.find("msg=")
            
            Data = GenerateRow(IndexKeys,Extension,Data)
            CustomLog.append(Data)
            
        elif 2000000<=SignatureID<=2999999:
            IndexKeys = dict()
            IndexKeys["cn1"] = Extension.find("cn1=")
            IndexKeys["cn1Label"] = Extension.find("cn1Label=")
            IndexKeys["dvc"]=Extension.find("dvc")
            IndexKeys["TrendMicroDsTenant"]=Extension.find("TrendMicroDsTenant")
            IndexKeys["TrendMicroDsTenantId"]=Extension.find("TrendMicroDsTenantId")
            IndexKeys["act"]=Extension.find("act")
            IndexKeys["filePath"]=Extension.find("filePath")
            IndexKeys["suser"]=Extension.find("suser")
            IndexKeys["sproc"]=Extension.find("sproc")
            IndexKeys["msg"]=Extension.find("msg")
           
            Data = GenerateRow(IndexKeys,Extension,Data)
            IntegrityMonitoring.append(Data)
            
        elif 3000000<=SignatureID<=3999999:
            IndexKeys = dict()
            IndexKeys["cn1"]=Extension.find("cn1=")
            IndexKeys["cn1Label"]=Extension.find("cn1Label=")
            IndexKeys["dvc"]=Extension.find("dvc=")
            IndexKeys["TrendMicroDsTenant"]=Extension.find("TrendMicroDsTenant=")
            IndexKeys["TrendMicroDsTenantId"]=Extension.find("TrendMicroDsTenantId=")
            IndexKeys["cs1"]=Extension.find("cs1=")
            IndexKeys["cs1Label"]=Extension.find("cs1Label=")
            IndexKeys["fname"]=Extension.find("fname=")
            IndexKeys["duser"]=Extension.find("duser=")
            IndexKeys["shost"]=Extension.find("shost=")
            IndexKeys["msg"]=Extension.find("msg=")
            Indexes = list(IndexKeys.values())
           
            Data = GenerateRow(IndexKeys,Extension,Data)
            LogInspection.append(Data)
    else:
        IndexKeys["src"]=Extension.find("src")
        IndexKeys["suser"]=Extension.find("suser")
        IndexKeys["target"]=Extension.find("target")
        IndexKeys["msg"]=Extension.find("msg")
        IndexKeys["TrendMicroDsTenant"]=Extension.find("TrendMicroDsTenant")
        IndexKeys["TrendMicroDsTenantID"]=Extension.find("TrendMicroDsTenantId")

        if IndexKeys["target"] == -1:
            IndexKeys["target"] = IndexKeys["msg"]

        Data = GenerateRow(IndexKeys,Extension,Data)
        Result.append(Data)

        

<h1> Helper functions </h1>
<ol>
    <li> WriteContent(Data : list , filename : string , header : list) -> Write Content to csv file.
    <li> GenerateRow(IndexKeys : dict , Extension : string , Data : list) -> Write Content to csv file.
</ol>

In [44]:
def WriteContent(Data,filename,header):
    """
    Write Data to text file
    
    with open(filename, 'w') as f:
        for item in Data:
            f.write("%s\n" % item)
    """
    with open(filename, "w", newline='') as f:
            writer = csv.writer(f)
            if header!=None:
                writer.writerow(header)
            for data in Data:
                writer.writerow(data)
def GenerateRow(IndexKeys,Extension,Data):
    Indexes = list(IndexKeys.values())
    for index in range(1,len(Indexes)):
        temp = Extension[Indexes[index - 1]:Indexes[index]].strip()
        Data.append(temp[temp.find("=")+1:])
    temp =  Extension[Indexes[-1]:].strip()
    Data.append(temp[temp.find("=")+1:])
    return Data
    

<h1> System Events </h1>

In [45]:
Header = ["Receiving Time","Generated Time","Service Facility","IPv4 Address","DeviceVendor","DeviceProduct","DeviceVersion","SignatureID","Name","Severity","src","suser","target","msg","TrendMicroDsTenant","TrendMicroDsTenantId"]
WriteContent(Result,"C:\PowerBI\Power_BI_Reports\DeepSecurity\SystemEventsNew.csv",Header)

<h1> Integrity Monitoring</h1>

In [46]:
IntegrityMonitoringHeader = ["Receiving Time","Generated Time","Service Facility","IPv4 Address","DeviceVendor","DeviceProduct","DeviceVersion","SignatureID","Name","Severity","Host ID","Host ID","Dvc","TrendMicroDsTenant","TrendMicroDsTenantId","Act","filePath","suser","sproc","msg"]
WriteContent(IntegrityMonitoring,"C:\\PowerBI\\Power_BI_Reports\DeepSecurity\\IntegrityMonitoringNew.csv",IntegrityMonitoringHeader)

<h1>Custom Log </h1>

In [47]:
### Custom Log New
CustomLogHeader = ["Receiving Time","Generated Time","Service Facility","IPv4 Address","DeviceVendor","DeviceProduct","DeviceVersion","SignatureID","Name","Severity","Host ID","Host ID","dvc","TrendMicroDsTenant","TrendMicroDsTenantId","LI Description","LI Description","fname","duser","shost","msg"]
WriteContent(CustomLog,"C:\\PowerBI\\Power_BI_Reports\DeepSecurity\\CustomLogNew.csv",CustomLogHeader)

<h1>Log Inspection</h1>

In [48]:

LogInspectionHeader = ["Receiving Time","Generated Time","Service Facility","IPv4 Address","DeviceVendor","DeviceProduct","DeviceVersion","SignatureID","Name","Severity","Host ID","Host ID","dvc","TrendMicroDsTenant","TrendMicroDsTenantId","LI Description","LI Description","fname","duser","shost","msg"]
WriteContent(LogInspection,"C:\\PowerBI\\Power_BI_Reports\DeepSecurity\\LogInspectionNew.csv",LogInspectionHeader)